In [70]:
from bs4 import BeautifulSoup
import requests
import json
import re

import pandas as pd

import os

In [68]:
def get_vehicle_info (soup):
    data_rows = []
    for row in soup.find_all('td', class_="Vehicle"):
        for td in row:
            car_url = url_base + td.get('href')
            names = [x.text for x in td.children]
            output = {}
            if ('Safety' in names[0]):
                make_model = names[1]
                years = ' '.join(names[2:])
                iihs_ranking = names[0]
            else:
                make_model = names[0]
                years = ' '.join(names[1:])
                iihs_ranking = ''
            make = make_model.split(' ')[0]
            model = ' '.join(make_model.split(' ')[1:])
            data_rows = data_rows + [{'make': make, 'model': model, 'years': years, "iihs_ranking": iihs_ranking, 'url': car_url}]

    df = pd.DataFrame(data_rows)
    return (df)

In [72]:
def extract_table (url):
    page = requests.get(url)
    df_raw = pd.read_html(page.text)[0]
    soup = BeautifulSoup(page.text, 'html.parser')
    df_id = get_vehicle_info(soup).assign(car_type=os.path.basename(url))
    df = pd.concat([df_id, df_raw.drop(columns="Vehicle Name")], axis=1)
    return (df)

In [37]:
car_types = ["small-cars", "midsize-cars", "midsize-luxury-cars", "large-cars", "large-luxury-cars", 
             "small-suvs", "midsize-suvs", "midsize-luxury-suvs", "large-suvs", "minivans"]
url_base = "https://www.iihs.org"
urls = [url_base + "/ratings/class-summary/" + x for x in car_types]

In [78]:
car_table = pd.concat([extract_table(x) for x in urls]).reset_index(drop=True)

# Save to file

In [ ]:
outdir = "./"

In [83]:
car_table.to_csv(os.path.join(outdir, "2022_iihs_car_rankings_raw_DO_NOT_EDIT.csv"))

In [84]:
car_table.to_excel(os.path.join(outdir, "2022_iihs_car_rankings.xlsx"), sheet_name="iihs car ranking")